In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from src import supabase_client
from src.update_pipeline import update_training_week
from src.types.update_pipeline import ExeType
from src import auth_manager
from src.utils import datetime_now_est, get_last_sunday

import os

# New Week Update

In [ ]:
user = supabase_client.get_user(os.environ["JAMIES_ATHLETE_ID"])
auth_manager.authenticate_athlete(user.athlete_id)
update_training_week(user, ExeType.NEW_WEEK, dt=get_last_sunday())

# Mid Week Update

In [ ]:
user = supabase_client.get_user(os.environ["JAMIES_ATHLETE_ID"])
auth_manager.authenticate_athlete(user.athlete_id)
update_training_week(user, ExeType.MID_WEEK, dt=datetime_now_est())

# Observability

In [1]:
import json
import polars as pl

with open("observe.jsonl", "r") as file:
    data = [json.loads(line) for line in file]

df = pl.DataFrame(data).drop([pl.col("messages"), pl.col("content")])

df

generation_name,response_id,model,completion_tokens,prompt_tokens,total_tokens,duration
str,str,str,i64,i64,i64,f64
"""gen_training_plan""","""chatcmpl-AvmV5n2ymD6LSN4bMmyCP…","""gpt-4o-2024-08-06""",437,1873,2310,6.560403
"""gen_coaches_notes""","""chatcmpl-AvmVFwECpx0DXIURhlWkr…","""gpt-4o-2024-08-06""",38,746,784,0.87857
"""gen_coaches_notes""","""chatcmpl-AvmVGthCHdwYKJKI9R5IT…","""gpt-4o-2024-08-06""",99,746,845,5.195081
"""gen_coaches_notes""","""chatcmpl-AvmVNtvZqpnTmjrNhknxm…","""gpt-4o-2024-08-06""",105,749,854,2.254943
"""gen_coaches_notes""","""chatcmpl-AvmVQkHzVqMBeSOW4u8ws…","""gpt-4o-2024-08-06""",47,754,801,1.229121
"""gen_coaches_notes""","""chatcmpl-AvmVSukdhFPz3Y00pcwBZ…","""gpt-4o-2024-08-06""",89,754,843,6.45575
"""gen_pseudo_training_week""","""chatcmpl-AvmVZDRFMaqzjernTHAEe…","""gpt-4o-2024-08-06""",52,1911,1963,3.222042
"""gen_training_week""","""chatcmpl-AvmVc8GqttGClF8mitUBA…","""gpt-4o-2024-08-06""",128,545,673,2.359972


In [2]:
df.group_by(pl.col("generation_name")).agg(pl.col('duration').sum())

generation_name,duration
str,f64
"""gen_coaches_notes""",16.013466
"""gen_training_week""",2.359972
"""gen_pseudo_training_week""",3.222042
"""gen_training_plan""",6.560403


In [3]:
df.sum().select(pl.col("duration"))

duration
f64
28.155882
